In [ ]:
import pandas as pd
import numpy as np
import matplotlib as plt
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
#!pip install scikit-learn==0.24.0

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import IncrementalPCA
from sklearn.linear_model import LogisticRegression

from sklearn.cluster import k_means
from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import MeanShift
from sklearn.cluster import OPTICS
from sklearn.cluster import DBSCAN
!pip install hdbscan
!pip install fa_kit
!pip install kneed
from fa_kit.rotation import VarimaxRotatorPython

from sklearn import metrics
import os

from sklearn.metrics import mean_squared_error, r2_score
from sklearn import datasets, linear_model

import glob
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors

from scipy.spatial import distance
from scipy.cluster import hierarchy

import seaborn as sns; sns.set();
from seaborn import heatmap

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error, r2_score
from sklearn import datasets, linear_model

import random as random
from sklearn.multiclass import OneVsRestClassifier

from xgboost import XGBClassifier

import itertools as it
from matplotlib.backends.backend_pdf import PdfPages
from sklearn.metrics import accuracy_score

from sklearn.neighbors import LocalOutlierFactor
from sklearn.neighbors import NearestNeighbors

In [2]:
def read_file(folder):
    df = pd.DataFrame()
    path = folder+'/'
    df_new = pd.read_csv(path+'benign_traffic.csv')
    df_new['label'] = 'benign_traffic'
    df = pd.concat([df,df_new], ignore_index=True)
    
    directory = os.fsencode(path+'gafgyt_attacks')
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        if filename.endswith(".csv"): 
            df_new = pd.read_csv(path+'gafgyt_attacks'+'/'+filename)
            filename=filename[:-4] 
            df_new['label'] = 'gafgyt_'+filename
            df = pd.concat([df,df_new], ignore_index=True)
    directory = os.fsencode(path+'mirai_attacks')
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        if filename.endswith(".csv"): 
            df_new = pd.read_csv(path+'mirai_attacks'+'/'+filename)
            filename=filename[:-4] 
            df_new['label'] = 'mirai_'+filename
            df = pd.concat([df,df_new], ignore_index=True)
    return df

In [2]:
#TODO: -scaling for lof


def read_file(folder, samp = False, lof=False):
    df = pd.DataFrame()
    path = folder+'/'
    df_new = pd.read_csv(path+'benign_traffic.csv')
    df_new['label'] = 'benign_traffic'
    if samp==True:
        df_new = df_new.sample(frac=0.5, replace=False, random_state=42).reset_index(drop=True)
    if lof==True:
        #df_new = pd.DataFrame(scaler.fit_transform(df_new))
        df_new = filter_lof(df_new)
        #df_new = scaler.inverse_transform(df_new)
    df = pd.concat([df,df_new], ignore_index=True)
    
    directory = os.fsencode(path+'gafgyt_attacks')
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        if filename.endswith(".csv"): 
            df_new = pd.read_csv(path+'gafgyt_attacks'+'/'+filename)
            filename=filename[:-4] 
            df_new['label'] = 'gafgyt_'+filename
            if samp==True:
                df_new = df_new.sample(frac=0.5, replace=False, random_state=42).reset_index(drop=True)
            if lof==True:
                #df_new = pd.DataFrame(scaler.fit_transform(df_new))
                df_new = filter_lof(df_new)
                #df_new = scaler.inverse_transform(df_new)
            df = pd.concat([df,df_new], ignore_index=True)
    directory = os.fsencode(path+'mirai_attacks')
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        if filename.endswith(".csv"): 
            df_new = pd.read_csv(path+'mirai_attacks'+'/'+filename)
            filename=filename[:-4] 
            df_new['label'] = 'mirai_'+filename
            if samp==True:
                df_new = df_new.sample(frac=0.5, replace=False, random_state=42).reset_index(drop=True)
            if lof==True:
                #df_new = pd.DataFrame(scaler.fit_transform(df_new))
                df_new = filter_lof(df_new)
                #df_new = scaler.inverse_transform(df_new)
            df = pd.concat([df,df_new], ignore_index=True)
    return df

In [4]:
#read in original data (wo anonymization)
def read_file_o(dfs, device):
    df = pd.DataFrame()
    for i in dfs:
        #df_new = pd.read_csv(i+'.csv')
        #df_new = pd.read_csv('./Simple_home_1003_sec_cam/'+i+'.csv')
        df_new = pd.read_csv('./'+device+'/'+i+'.csv')
        df_new['label'] = i
        df = pd.concat([df,df_new], ignore_index=True)
    return df

In [5]:
def read_file_new(dfs, device, lof = False):
    df = pd.DataFrame()
    for i in dfs:
        df_new = pd.read_csv("/data/dataprivacy/detection_of_IoT_botnet_attacks/"+device+'/'+i+'.csv')
        #df_new = pd.read_csv('./'+device+'/'+i+'.csv')
        df_new['label'] = i
        df_new = df_new.sample(frac=0.5, replace=False, random_state=42).reset_index(drop=True)
        if lof==True:
            df_new = filter_lof(df_new)
        df = pd.concat([df,df_new], ignore_index=True)
    return df

In [3]:
def filter_lof(df, k=20) :
    lof = LocalOutlierFactor(n_neighbors=k)
    scaler = StandardScaler()
    df2 = pd.DataFrame.copy(df)
    df2 = df2.drop(columns = ['label'])
    df2 = pd.DataFrame(scaler.fit_transform(df2))
    df2["_lof"] = lof.fit_predict(df2)
    df2 = df2[df2["_lof"]>0].drop(columns="_lof")#.reset_index(drop=True)
    df2 = pd.DataFrame(scaler.inverse_transform(df2))
    df2['label'] = df['label']
    return df2.reset_index(drop=True)

In [ ]:
#read original data

files = ['benign_traffic', 'bashlite_combo', 'bashlite_junk', 'bashlite_scan', 'bashlite_tcp', 'bashlite_udp', 'mirai_ack', 'mirai_scan', 'mirai_syn', 'mirai_udp', 'mirai_udpplain']
df_original = read_file_new(files,'SimpleHome_XCS7_1003_WHT_Security_Camera')
#df_original = filter_lof(df_original)
#df_original = df_original.sample(frac=0.5, replace=False, random_state=42).reset_index(drop=True)
#df_original = df_original.dropna().reset_index(drop=True)
#df_original 

#nbrs = NearestNeighbors(n_neighbors=2, algorithm='ball_tree').fit(df_original)

#nbrs.kneighbors_graph(X).toarray()

In [4]:
def crop_attila(df, cluster_size):
    firsts = []
    files = df['label'].unique()
    to_delete_list = []
    for i in range(len(files)):
        #last type
        if i==len(files)-1:
            first = len(df)
            firsts.append(first)
        else:
            first = df.index[df['label']==files[i+1]].tolist()[0]
            firsts.append(first)
        #first type
        if i==0:
            to_delete = first%cluster_size
        else:
            to_delete = (first-firsts[i-1])%cluster_size
        if to_delete==0 and files[i]=='mirai_syn':
            to_delete=cluster_size
        if to_delete==0 and files[i]=='gafgyt_tcp':
            to_delete=cluster_size
        #chosing rows to drop
        for j in range(to_delete):
            to_delete_list.append(first-j-1)
    df = df.drop(to_delete_list)
    df = df.reset_index(drop=True)
    return df

In [14]:
df_original = read_file('/data/dataprivacy/detection_of_IoT_botnet_attacks/'+'SimpleHome_XCS7_1003_WHT_Security_Camera', samp=True, lof=False)
file = 'c10_e1'
df_original = crop_attila(df_original, int(file[1:3]))
print(len(df_original))

4
9
6
8
6
0
10
4
7
8
2
425350


In [11]:
for i in df_original['label'].unique():
    print(i)
    print(df_original[df_original['label'] == i].shape[0])

benign_traffic
9760
gafgyt_combo
29690
gafgyt_junk
13700
gafgyt_tcp
49030
gafgyt_scan
14280
gafgyt_udp
51490
mirai_syn
61230
mirai_ack
53590
mirai_scan
21830
mirai_udpplain
42210
mirai_udp
78540


In [12]:
for i in df['label'].unique():
    print(i)
    print(df[df['label'] == i].shape[0])

benign_traffic
9760
gafgyt_combo
29690
gafgyt_junk
13700
gafgyt_tcp
49030
gafgyt_scan
14280
gafgyt_udp
51490
mirai_syn
61230
mirai_ack
53590
mirai_scan
21830
mirai_udpplain
42210
mirai_udp
78540


In [15]:
file = 'c10_e1'
df = read_file('/data/dataprivacy/detection_of_IoT_botnet_attacks-anonymized/'+'SimpleHome_XCS7_1003_WHT_Security_Camera'+'/'+'attila1'+'/'+file)
#df = df.sample(frac=0.5, replace=False, random_state=42).reset_index(drop=True)
#df = df.dropna().reset_index(drop=True)
print(len(df))

425350


In [ ]:
df

In [ ]:
df[df['label']=='mirai_syn']

In [ ]:
df['label'].unique()

# k nearest neighbor

In [5]:
from sklearn.neighbors import NearestNeighbors
import numpy as np

In [25]:
def eval_privacy(devices, algos):
    df_avg_ident = pd.DataFrame(columns = ['avg_per_10'])
    for device in devices:
        print(device)
        for algo in algos:
            print(algo)
            if algo=='attila1' or algo=='attila2':
                df_original = read_file('/data/dataprivacy/detection_of_IoT_botnet_attacks/'+device, samp=True, lof=False)
                print('attila')
            else:
                df_original = read_file('/data/dataprivacy/detection_of_IoT_botnet_attacks/'+device, samp=True, lof=True)
                print('nemattila')
            for file in os.listdir('/data/dataprivacy/detection_of_IoT_botnet_attacks-anonymized/'+device+'/'+algo):
                if file.startswith('c') or file.startswith('k'):
                    print(file)
                    if algo=='attila1' or algo=='attila2':
                        if int(file[1])==3 or int(file[1])==5:
                            num = int(file[1:2])
                        else:
                            num = int(file[1:3])
                        print(num)
                        df_original_cropped = crop_attila(df_original, num)
                        df = read_file('/data/dataprivacy/detection_of_IoT_botnet_attacks-anonymized/'+device+'/'+algo+'/'+file)
                        print(len(df_original_cropped))
                        print(len(df))
                        for j in df_original_cropped['label'].unique():
                            if df_original_cropped[df_original_cropped['label'] == j].shape[0] != df[df['label'] == j].shape[0]:
                                print('baaaj')
                                print(j)
                                print(df_original_cropped[df_original_cropped['label'] == j].shape[0])
                                print(df[df['label'] == j].shape[0])
                        X = df_original_cropped.iloc[:,:-1]
                    else:
                        df = read_file('/data/dataprivacy/detection_of_IoT_botnet_attacks-anonymized/'+device+'/'+algo+'/'+file)
                        print(len(df_original))
                        print(len(df))
                        for j in df_original['label'].unique():
                            if df_original[df_original['label'] == j].shape[0] != df[df['label'] == j].shape[0]:
                                print('baaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaj')
                                print(j)
                                print(df_original[df_original['label'] == j].shape[0])
                                print(df[df['label'] == j].shape[0])
                        X = df_original.iloc[:,:-1]
                    nbrs_original = NearestNeighbors(n_neighbors=11, algorithm='ball_tree').fit(X)
                    distances_original, indices_original = nbrs_original.kneighbors(X)
                    indices_original = indices_original[:,1:]
                    inds = pd.isnull(df).any(1).to_numpy().nonzero()
                    df = df.dropna().reset_index(drop=True)
                    X = X.drop(inds[0]).reset_index(drop=True)
                    Y = df.iloc[:,:-1]
                    nbrs = NearestNeighbors(n_neighbors=11, algorithm='ball_tree').fit(Y)
                    distances, indices = nbrs.kneighbors(Y)
                    indices = indices[:,1:]
                    identical = []
                    for l in range(len(indices)):
                        identical.append(len(set(indices_original[l]).intersection(indices[l])))
                    df_avg_ident.loc[device+'_'+algo+'_'+file] = sum(identical)/len(identical)
                    df_avg_ident.to_csv('eval_privacy_seccam.csv')

In [26]:
devices = ['SimpleHome_XCS7_1003_WHT_Security_Camera']
algos = ['alex1b','alex2']
eval_privacy(devices, algos)

SimpleHome_XCS7_1003_WHT_Security_Camera
alex1b
nemattila
k10_e10_lof
396042
396042
k10_e1_lof
396042
396042
k20_e5_lof
396042
396042
k20_e1_lof
396042
396042
k50_e5_lof
396042
396042
k50_e1_lof
396042
396042
k20_e10_lof
396042
396042
k10_e5_lof
396042
396042
k50_e10_lof
396042
396042
alex2
nemattila
k10_e10_lof
396042
396042
k10_e1_lof
396042
396042
k20_e5_lof
396042
396042
k20_e1_lof
396042
396042
k50_e5_lof
396042
396042
k50_e1_lof
396042
396042
k20_e10_lof
396042
396042
k10_e5_lof
396042
396042
k50_e10_lof
396042
396042


In [ ]:
len(indices_original)

In [ ]:
df_original = read_file('/data/dataprivacy/detection_of_IoT_botnet_attacks/'+'SimpleHome_XCS7_1003_WHT_Security_Camera', samp=True, lof=True)

In [ ]:
file = 'k10_e1_lof'
df = read_file('/data/dataprivacy/detection_of_IoT_botnet_attacks-anonymized/'+'SimpleHome_XCS7_1003_WHT_Security_Camera'+'/'+'alex2'+'/'+file)

In [ ]:
X = df_original.iloc[:,:-1]
nbrs_original = NearestNeighbors(n_neighbors=11, algorithm='ball_tree').fit(X)
distances_original, indices_original = nbrs_original.kneighbors(X)
indices_original = indices_original[:,1:]

In [ ]:
Y = df.iloc[:,:-1]
nbrs = NearestNeighbors(n_neighbors=11, algorithm='ball_tree').fit(Y)
distances, indices = nbrs.kneighbors(Y)
indices = indices[:,1:]

In [ ]:
identical = []
for i in range(len(indices)):
    identical.append(len(set(indices_original[i]).intersection(indices[i])))

In [ ]:
print(sum(identical)/len(identical))